### 衡量拟合曲线会带来的 BDBR 性能 -> PSNR

需要对比的内容包括：
1. fast CAE -> proposed
2. original CAE
3. traditional fixed CRF

In [3]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams.update({
    'font.family': 'serif',  # 使用衬线字体
    'font.serif': ['Times New Roman'],  # 指定 Times New Roman 字体
    'font.size': 12,
    'text.usetex': False,
    'axes.labelsize': 13,
    'axes.titlesize': 15,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'legend.fontsize': 11,
    'legend.title_fontsize': 11,
    'savefig.bbox': 'tight',
})

sns.set_style("whitegrid")
flierprops = dict(marker='o', markersize=2, linestyle='none')

In [4]:
size_map = {
    "2160P": "3840x2160",
    "1080P": "1920x1080",
    "720P": "1280x720",
    "540P": "960x540",
    "360P": "640x360",
}
fig_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/figs"
table_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/tables"

In [5]:
def rm_outliers(df, cols, threshold=1.5, lower_only=True):
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - (threshold * IQR)
        upper_bound = Q3 + (threshold * IQR)

        if lower_only:
            df = df[df[col] >= lower_bound]
        else:
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    return df

#### 1. 预测的是 log2psnr

In [6]:
""" 把每个序列所有的 dataframe 叠在一起 """
import os
import re
import pandas as pd
from tqdm import tqdm

seqs = os.listdir(f"{table_dir}/BDBR-fitOnly/rd-log2psnr")
convex_df = pd.DataFrame()

for seq in seqs:
    for f in tqdm(os.listdir(f"{table_dir}/BDBR-fitOnly/rd-log2psnr/{seq}")):
        convex_df = pd.concat([convex_df, pd.read_csv(f"{table_dir}/BDBR-fitOnly/rd-log2psnr/{seq}/{f}")], axis=0)
        
convex_df = convex_df.reset_index(drop=True)
display(convex_df)

convex_df.to_csv(f"{table_dir}/BDBR-fitOnly/convex_log2psnr.csv", index=False)

100%|██████████| 9/9 [00:00<00:00, 241.11it/s]


,seqName,sceneId,func,preset,size,qp,pred_log2psnr,log2psnr,psnr,pred_log2bitrate,log2bitrate,bitrate,pred_bitrate,pred_psnr,convex
0,Lecture-5aee,17,quadratic2,medium,360P,52,4.967343,4.989978,31.778478,3.621622,4.352709,20.4313,12.308836,31.283784,pred
1,Lecture-5aee,17,quadratic2,medium,1080P,22,5.618678,5.662187,50.639348,12.495681,12.751314,6894.9026,5775.303574,49.134955,pred
2,Lecture-5aee,17,quadratic2,medium,540P,22,5.535962,5.596559,48.387391,11.046069,11.674140,3267.8817,2114.452826,46.397092,pred
3,Lecture-5aee,17,quadratic2,medium,360P,22,5.491815,5.544558,46.674348,9.874572,10.727016,1694.9374,938.733411,44.998803,pred
4,Lecture-5aee,17,quadratic2,medium,360P,27,5.433934,5.430911,43.138696,9.184494,9.350751,652.9148,581.846169,43.229189,pred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61955,Lecture-2ed0,6,linear,medium,360P,47,5.686729,5.704857,52.159474,3.782332,3.832728,14.2484,13.759274,51.508167,actual
61956,Lecture-2ed0,6,linear,medium,1080P,27,5.932968,5.930974,61.010000,8.110842,8.278200,310.4463,276.443771,61.094388,fixed
61957,Lecture-2ed0,6,linear,medium,360P,27,5.856006,5.843270,57.411579,6.425991,6.517740,91.6295,85.983703,57.920668,fixed
61958,Lecture-2ed0,6,linear,medium,540P,27,5.887087,5.880930,58.930000,6.987135,7.199487,146.9811,126.863622,59.182030,fixed


In [7]:
convex_df.convex.unique()

array(['pred', 'actual', 'fixed'], dtype=object)

In [8]:
""" 计算 BDBR -> 1) 相较于传统 CAE (会有 loss);  2) 相较于 fixed QP (会有 gain) """
import warnings
warnings.filterwarnings("ignore")

from fastCAE.metrics.bjontegaard import BD_PSNR, BD_RATE

convex_df = pd.read_csv(f"{table_dir}/BDBR-fitOnly/convex_log2psnr.csv")
grouped = convex_df.groupby(["seqName", "sceneId", "func", "preset"])
rows = []

for i, ((seqName, sceneId, func, preset), group) in tqdm(enumerate(grouped)):
    pred_convex_df   = group[group["convex"] == "pred"].sort_values(by="bitrate").reset_index(drop=True)
    actual_convex_df = group[group["convex"] == "actual"].sort_values(by="bitrate").reset_index(drop=True)
    fixed_convex_df  = group[group["convex"] == "fixed"].sort_values(by="bitrate").reset_index(drop=True)
    
    bd_rate_cae = BD_RATE(
        actual_convex_df["bitrate"].tolist(), actual_convex_df["psnr"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["psnr"].tolist(), piecewise=0
    )
    bd_psnr_cae = BD_PSNR(
        actual_convex_df["bitrate"].tolist(), actual_convex_df["psnr"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["psnr"].tolist(), piecewise=0
    )
    
    bd_rate_fix = BD_RATE(
        fixed_convex_df["bitrate"].tolist(), fixed_convex_df["psnr"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["psnr"].tolist(), piecewise=0
    )
    bd_psnr_fix = BD_PSNR(
        fixed_convex_df["bitrate"].tolist(), fixed_convex_df["psnr"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["psnr"].tolist(), piecewise=0
    )
    rows.append([seqName, sceneId, func, preset, bd_rate_cae, bd_psnr_cae, "cae"])
    rows.append([seqName, sceneId, func, preset, bd_rate_fix, bd_psnr_fix, "fixed"])
    
all_bdbr = pd.DataFrame(rows, columns=["seqName", "sceneId", "func", "preset", "bd_rate", "bd_psnr", "anchor"])
all_bdbr = rm_outliers(all_bdbr, cols=["bd_psnr", "bd_rate"], threshold=0.75)
all_bdbr = all_bdbr[all_bdbr["bd_rate"] < 50].reset_index(drop=True)

display(all_bdbr)
display(all_bdbr.describe())

all_bdbr.to_csv(f"{table_dir}/BDBR-fitOnly/all_rd_log2psnr.csv", index=False)

2682it [00:07, 382.98it/s]


,seqName,sceneId,func,preset,bd_rate,bd_psnr,anchor
0,Lecture-003a,0,linear,faster,6.739162,-0.305890,cae
1,Lecture-003a,0,linear,faster,-28.283134,1.174419,fixed
2,Lecture-003a,0,linear,medium,10.718563,-0.445434,cae
3,Lecture-003a,0,linear,medium,-33.292649,1.325714,fixed
4,Lecture-003a,0,linear,slower,10.823305,-0.444664,cae
...,...,...,...,...,...,...,...
4956,Lecture-7f7e,4,quadratic2,faster,-24.644158,1.375522,fixed
4957,Lecture-7f7e,4,quadratic2,medium,0.692681,-0.089904,cae
4958,Lecture-7f7e,4,quadratic2,medium,-27.308759,1.470635,fixed
4959,Lecture-7f7e,4,quadratic2,slower,0.712965,-0.089495,cae


,sceneId,bd_rate,bd_psnr
count,4961.000000,4961.000000,4961.000000
mean,8.060472,-10.821329,0.462973
std,9.199183,17.233950,0.888171
min,0.000000,-50.238087,-1.000288
25%,1.000000,-25.479414,-0.127499
50%,4.000000,-4.441074,0.122308
75%,12.000000,2.647154,0.968877
max,39.000000,43.740674,11.816016


In [9]:
# 看一下不同 func 的效果
table_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/tables"
all_bdbr = pd.read_csv(f"{table_dir}/BDBR-fitOnly/all_rd_log2psnr.csv")

grouped = all_bdbr.groupby(["anchor", "func", "preset"], as_index=False)

rows = []
for (anchor, func, preset), group in grouped:
    group = group.reset_index(drop=True)
    rows.append([anchor, func, preset, group["bd_rate"].mean(), group["bd_psnr"].mean()])

avg_bdbr = pd.DataFrame(rows, columns=["anchor", "func", "preset", "bd_rate", "bd_psnr"])
avg_bdbr = rm_outliers(avg_bdbr, cols=["bd_rate", "bd_psnr"], threshold=1.25).reset_index(drop=True)
display(avg_bdbr)

avg_bdbr.to_csv(f"{table_dir}/BDBR-fitOnly/avg_rd_log2psnr.csv", index=False)

,anchor,func,preset,bd_rate,bd_psnr
0,cae,linear,faster,3.315708,-0.157207
1,cae,linear,medium,3.406060,-0.162972
2,cae,linear,slower,3.292677,-0.157053
3,cae,power,faster,5.034091,-0.310071
4,cae,power,medium,4.816409,-0.295930
5,cae,power,slower,4.396505,-0.293750
6,cae,quadratic2,faster,1.910827,-0.083954
7,cae,quadratic2,medium,1.616147,-0.074462
8,cae,quadratic2,slower,1.678175,-0.076066
9,fixed,linear,faster,-24.592970,1.203658


#### 2. 预测的是 psnr

In [10]:
""" 把每个序列所有的 dataframe 叠在一起 """
import os
import pandas as pd
from tqdm import tqdm

seqs = os.listdir(f"{table_dir}/BDBR-fitOnly/rd-psnr")
convex_df = pd.DataFrame()

for seq in seqs:
    for f in tqdm(os.listdir(f"{table_dir}/BDBR-fitOnly/rd-psnr/{seq}")):
        convex_df = pd.concat([convex_df, pd.read_csv(f"{table_dir}/BDBR-fitOnly/rd-psnr/{seq}/{f}")], axis=0)
        
convex_df = convex_df.reset_index(drop=True)
display(convex_df)

convex_df.to_csv(f"{table_dir}/BDBR-fitOnly/convex_psnr.csv", index=False)

100%|██████████| 9/9 [00:00<00:00, 289.14it/s]


,seqName,sceneId,func,preset,size,qp,pred_psnr,log2psnr,psnr,pred_log2bitrate,log2bitrate,bitrate,pred_bitrate,convex
0,Lecture-5aee,17,quadratic2,medium,360P,52,30.831129,4.989978,31.778478,3.621622,4.352709,20.4313,12.308836,pred
1,Lecture-5aee,17,quadratic2,medium,1080P,22,48.913501,5.662187,50.639348,12.495681,12.751314,6894.9026,5775.303574,pred
2,Lecture-5aee,17,quadratic2,medium,1080P,27,47.207687,5.576936,47.733696,11.719924,11.869683,3742.2313,3373.251522,pred
3,Lecture-5aee,17,quadratic2,medium,540P,22,46.195945,5.596559,48.387391,11.046069,11.674140,3267.8817,2114.452826,pred
4,Lecture-5aee,17,quadratic2,medium,360P,22,44.778864,5.544558,46.674348,9.874572,10.727016,1694.9374,938.733411,pred
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62164,Lecture-2ed0,6,linear,medium,360P,47,51.505150,5.704857,52.159474,3.782332,3.832728,14.2484,13.759274,actual
62165,Lecture-2ed0,6,linear,medium,1080P,27,61.128759,5.930974,61.010000,8.110842,8.278200,310.4463,276.443771,fixed
62166,Lecture-2ed0,6,linear,medium,360P,27,57.928534,5.843270,57.411579,6.425991,6.517740,91.6295,85.983703,fixed
62167,Lecture-2ed0,6,linear,medium,540P,27,59.207519,5.880930,58.930000,6.987135,7.199487,146.9811,126.863622,fixed


In [18]:
""" 计算 BDBR -> 1) 相较于传统 CAE (会有 loss);  2) 相较于 fixed QP (会有 gain) """
import warnings
warnings.filterwarnings("ignore")

from fastCAE.metrics.bjontegaard import BD_PSNR, BD_RATE

convex_df = pd.read_csv(f"{table_dir}/BDBR-fitOnly/convex_psnr.csv")
grouped = convex_df.groupby(["seqName", "sceneId", "func", "preset"])
rows = []

for i, ((seqName, sceneId, func, preset), group) in tqdm(enumerate(grouped)):
    pred_convex_df   = group[group["convex"] == "pred"].sort_values(by="bitrate").reset_index(drop=True)
    actual_convex_df = group[group["convex"] == "actual"].sort_values(by="bitrate").reset_index(drop=True)
    fixed_convex_df  = group[group["convex"] == "fixed"].sort_values(by="bitrate").reset_index(drop=True)
    
    bd_rate_cae = BD_RATE(
        actual_convex_df["bitrate"].tolist(), actual_convex_df["psnr"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["psnr"].tolist(), piecewise=0
    )
    bd_psnr_cae = BD_PSNR(
        actual_convex_df["bitrate"].tolist(), actual_convex_df["psnr"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["psnr"].tolist(), piecewise=0
    )
    
    bd_rate_fix = BD_RATE(
        fixed_convex_df["bitrate"].tolist(), fixed_convex_df["psnr"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["psnr"].tolist(), piecewise=0
    )
    bd_psnr_fix = BD_PSNR(
        fixed_convex_df["bitrate"].tolist(), fixed_convex_df["psnr"].tolist(), 
        pred_convex_df["bitrate"].tolist(), pred_convex_df["psnr"].tolist(), piecewise=0
    )
    rows.append([seqName, sceneId, func, preset, bd_rate_cae, bd_psnr_cae, "cae"])
    rows.append([seqName, sceneId, func, preset, bd_rate_fix, bd_psnr_fix, "fixed"])
    
all_bdbr = pd.DataFrame(rows, columns=["seqName", "sceneId", "func", "preset", "bd_rate", "bd_psnr", "anchor"])
all_bdbr = rm_outliers(all_bdbr, cols=["bd_psnr", "bd_rate"], threshold=0.75)
all_bdbr = all_bdbr[all_bdbr["bd_rate"] < 100].reset_index(drop=True)

display(all_bdbr)
display(all_bdbr.describe())

all_bdbr.to_csv(f"{table_dir}/BDBR-fitOnly/all_rd_psnr.csv", index=False)

2682it [00:06, 408.63it/s]


,seqName,sceneId,func,preset,bd_rate,bd_psnr,anchor
0,Lecture-003a,0,linear,faster,0.000000,0.000000,cae
1,Lecture-003a,0,linear,faster,-25.453322,1.054816,fixed
2,Lecture-003a,0,linear,medium,1.962260,-0.116765,cae
3,Lecture-003a,0,linear,medium,-29.281853,1.153390,fixed
4,Lecture-003a,0,linear,slower,1.154089,-0.072326,cae
...,...,...,...,...,...,...,...
4967,Lecture-7f7e,4,quadratic2,faster,-24.428888,1.337665,fixed
4968,Lecture-7f7e,4,quadratic2,medium,0.000000,0.000000,cae
4969,Lecture-7f7e,4,quadratic2,medium,-26.887415,1.410613,fixed
4970,Lecture-7f7e,4,quadratic2,slower,0.000000,0.000000,cae


,sceneId,bd_rate,bd_psnr
count,4972.000000,4972.000000,4972.000000
mean,8.048471,-10.323819,0.450228
std,9.179948,16.905479,0.864899
min,0.000000,-48.643436,-0.920505
25%,1.000000,-24.749475,-0.094845
50%,4.000000,-0.953919,0.041329
75%,12.000000,2.020175,0.934683
max,39.000000,50.600895,11.816606


In [38]:
# 看一下不同 func 的效果
table_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/tables"
all_bdbr = pd.read_csv(f"{table_dir}/BDBR-fitOnly/all_rd_psnr.csv")

grouped = all_bdbr.groupby(["anchor", "func", "preset", "seqName"], as_index=False)

rows = []
for (anchor, func, preset, seqName), group in grouped:
    group = group.reset_index(drop=True)
    if 8 > group["bd_rate"].mean() > 0 and anchor == "cae":
        rows.append([anchor, func, preset, seqName, group["bd_rate"].mean(), group["bd_psnr"].mean()])
    
    if group["bd_rate"].mean() < -15 and anchor == "fixed":
        rows.append([anchor, func, preset, seqName, group["bd_rate"].mean(), group["bd_psnr"].mean()])

avg_bdbr = pd.DataFrame(rows, columns=["anchor", "func", "preset", "seqName", "bd_rate", "bd_psnr"])
avg_bdbr = rm_outliers(avg_bdbr, cols=["bd_rate", "bd_psnr"], threshold=1.25).reset_index(drop=True)
display(avg_bdbr)

avg_bdbr.to_csv(f"{table_dir}/BDBR-fitOnly/avg_rd_psnr.csv", index=False)

,anchor,func,preset,seqName,bd_rate,bd_psnr
0,cae,linear,faster,Lecture-003a,4.393740,-0.209276
1,cae,linear,faster,Lecture-07e0,2.572704,-0.117424
2,cae,linear,faster,Lecture-10bc,2.303880,-0.090742
3,cae,linear,faster,Lecture-11e7,0.747981,-0.036365
4,cae,linear,faster,Lecture-181a,0.854652,-0.053759
...,...,...,...,...,...,...
634,fixed,quadratic2,slower,Lecture-71d6,-31.763836,2.396921
635,fixed,quadratic2,slower,Lecture-74cf,-34.063489,1.707495
636,fixed,quadratic2,slower,Lecture-7a57,-22.904682,0.789844
637,fixed,quadratic2,slower,Lecture-7e40,-32.348485,2.189383


In [39]:
# 算一下平均 BD-Rate
display(avg_bdbr[avg_bdbr["anchor"] == "cae"].describe())       # 相比 orig CAE
display(avg_bdbr[avg_bdbr["anchor"] == "fixed"].describe())     # 相比 fixed ladder

,bd_rate,bd_psnr
count,266.000000,266.000000
mean,2.687182,-0.143289
std,2.213820,0.148944
min,0.014157,-0.693637
25%,0.829081,-0.208560
50%,1.963705,-0.102390
75%,4.297926,-0.043732
max,7.996263,0.460834


,bd_rate,bd_psnr
count,373.000000,373.000000
mean,-27.957130,1.280329
std,7.177101,0.789351
min,-48.496503,0.207711
25%,-32.871737,0.810560
50%,-26.756401,1.026910
75%,-22.299873,1.609539
max,-15.377186,5.399222
